In [ ]:
import folium
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
import time
from IPython.display import clear_output
import threading
import websocket
import time
from IPython.display import display, clear_output


In [ ]:
def initialize_map():
    carte = folium.Map(location=[48.8566, 2.3522], zoom_start=11)
    return carte

# Create the initial map
carte = initialize_map()

In [ ]:
def creer_carte_folium(journeys, current_indices):
    # Créer une carte centrée sur Paris
    carte = folium.Map(location=[48.8566, 2.3522], zoom_start=11)

    # Dictionnaire pour stocker les éléments de la légende
    legend_html = {}

    # Créer une palette de couleurs pour chaque ligne
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']
    color_map = {line: colors[i % len(colors)] for i, line in enumerate(journeys.keys())}

    for line, gdf_journeys in journeys.items():
        line_color = color_map[line]
        for j, gdf_journey in enumerate(gdf_journeys):
            current_index = current_indices[line][j]

            # Créer une liste de coordonnées pour la ligne complète
            locations = [(lat, lon) for lon, lat in zip(gdf_journey.geometry.x, gdf_journey.geometry.y)]
            folium.PolyLine(locations, weight=2, color=line_color, opacity=0.5, dash_array="5, 5", popup=f"RER {line} Path").add_to(carte)

            # Créer une liste de coordonnées pour la progression actuelle
            current_locations = locations[:current_index + 1]

            # Vérifier si current_locations a au moins deux coordonnées
            if len(current_locations) >= 2:
                folium.PolyLine(current_locations, weight=4, color=line_color, popup=f"RER {line} Progress").add_to(carte)

                # Add a marker for the current train position
                current_position = current_locations[-1]
                folium.CircleMarker(current_position, radius=5, fill=True, color=line_color, fillColor=line_color, popup=f"RER {line}").add_to(carte)

            # Add markers for the stations
            for idx, row in gdf_journey.iterrows():
                station_label = f"{line}{idx + 1}"
                folium.CircleMarker(
                    [row.geometry.y, row.geometry.x],
                    radius=3,
                    fill=True,
                    color=line_color,
                    fillColor='white',
                    popup=f"{station_label}: {row['stop_point_name']}"
                ).add_to(carte)

        # Ajouter un élément à la légende pour la ligne actuelle
        legend_html[line] = f'<span style="color:{line_color};"><i class="fa fa-circle"></i></span> RER {line}'

    # Créer le HTML de la légende
    legend_html = '<div style="position: fixed; bottom: 50px; left: 50px; width: 150px; height: 100px; background-color: white; padding: 10px; border-radius: 5px; z-index: 1000; overflow-y: auto;">' + '<br>'.join(legend_html.values()) + '</div>'

    # Ajouter la légende à la carte
    carte.get_root().html.add_child(folium.Element(legend_html))

    return carte

In [ ]:

dataframes = []
journeys = {}
def add_data_to_map(message: str):
    print("Ajout de données à la carte")
    print(f"Message : {message}")
    
    # Split the message into 5 lines
    lines = message.split('\n')
    if len(lines) != 1:
        print(f"Erreur : 1 lignes attendues, {len(lines)} reçues")
        return
    
    columns = ["stop_point_id", "journey_id", "arrival_time", "departure_time", "stop_point_name", "longitude", "latitude", "rer_line"]
    
    for line in lines:
        # Diviser la ligne en utilisant la virgule comme séparateur
        data = line.split(',')
        
        # S'assurer que nous avons le bon nombre de colonnes
        if len(data) != len(columns):
            print(f"Erreur : {len(columns)} columns attendues, données reçues ont {len(data)} colonnes")
            continue
        
        df = pd.DataFrame([data], columns=columns)
        
        # Convertir les colonnes longitude et latitude en float
        df['longitude'] = df['longitude'].astype(float)
        df['latitude'] = df['latitude'].astype(float)
        
        dataframes.append(df)

    # Concatenate all DataFrames into a single DataFrame
    train_data = pd.concat(dataframes, ignore_index=True)

    # Group the data by journey_id and rer_line
    grouped_data = train_data.groupby(['journey_id', 'rer_line'])

    for (journey_id, line), journey in grouped_data:
        geometry = [Point(xy) for xy in zip(journey["longitude"], journey["latitude"])]
        gdf_journey = gpd.GeoDataFrame(journey, geometry=geometry)

        # Add the GeoDataFrame to the corresponding list in journeys
        if line in journeys:
            journeys[line].append(gdf_journey)
        else:
            journeys[line] = [gdf_journey]

    # Update the map
    current_indices = {l: [len(gdf) - 1 for gdf in gdf_journeys] for l, gdf_journeys in journeys.items()}
    carte = creer_carte_folium(journeys, current_indices)
    
    # Clear the output and display the updated map
    clear_output(wait=True)
    display(carte)

In [ ]:
def on_message(ws, message):
    print("Message reçu")
    print(f"Message reçu : {message}")
    add_data_to_map(message)
    
def on_error(ws, error):
    print(f"Erreur : {error}")


def on_close(ws, close_status_code, close_msg):
    print("Connexion fermée")


def on_open(ws):
    print("Connexion ouverte")


def run_websocket():
    global carte
    print("Starting WebSocket client...")
    ws = websocket.WebSocketApp(
        "ws://localhost:8888",
        on_open=on_open,
        on_message=on_message,
        on_error=on_error,
        on_close=on_close,
    )

    # Initialize and display the map
    carte = initialize_map()
    display(carte)

    # Run the WebSocket client in a thread
    wst = threading.Thread(target=ws.run_forever)
    wst.daemon = True
    wst.start()

    try:
        while True:
            time.sleep(1)  # Sleep to prevent high CPU usage
    except KeyboardInterrupt:
        print("Interruption par l'utilisateur")
        ws.close()
        wst.join()  # Ensure the thread finishes before exiting

In [ ]:
run_websocket()